In [1]:
## setup
!pip install ../input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl

Processing /kaggle/input/nnaudio031/nnAudio-0.3.1-py3-none-any.whl


In [2]:
# !ls /kaggle/input/blef0503/
# !ls /kaggle/input/ef-v0/
# !ls /kaggle/input/birds-inference-pip-wheels/*
# !pip freeze|grep torch
!ls /kaggle/input/ef-v10

ef_v10TimmClassifier_v3_tf_efficientnet_b0_0_lb
ef_v10TimmClassifier_v3_tf_efficientnet_b0_10_lb
ef_v10TimmClassifier_v3_tf_efficientnet_b0_1_lb
ef_v10TimmClassifier_v3_tf_efficientnet_b0_2_lb
ef_v10TimmClassifier_v3_tf_efficientnet_b0_3_lb
ef_v10TimmClassifier_v3_tf_efficientnet_b0_4_lb


In [3]:
## https://github.com/Selimonder/birdclef2022/
import os, sys, glob, math

os.environ["MKL_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"] = "1"

github_folder = "/kaggle/input/blef0508v3"
sys.path.append(github_folder)

In [4]:
import cv2
import gc
import time
import torch
import librosa
import argparse, warnings

import numpy as np
import pandas as pd
import IPython.display as ipd
from pathlib import Path


import zoo
from training.config import load_config
from training.datasets import BirdDatasetOOF


cv2.ocl.setUseOpenCL(False)
cv2.setNumThreads(0)

from tqdm import tqdm
from torch.utils.data import DataLoader

warnings.simplefilter("ignore")


/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/nnAudio/Spectrogram.py:4: Warning: importing Spectrogram subpackage will be deprecated soon. You should import the feature extractor from the feature subpackage. See actual documentation.
  warnings.warn(


In [5]:
# accelerate
import os
os.environ["OMP_NUM_THREADS"]="2"
os.environ["OMP_SCHEDULE"]="STATIC"
os.environ["OMP_PROC_BIND"]="CLOSE"
import torch
torch.set_num_threads(4)

## models

In [6]:
def load_model(conf_path: str, weights_path: str, prefix: str, suffix: str, fold: int, to_device: bool = True):
    conf = load_config(conf_path)
    conf['encoder_params']['pretrained'] = False
    print(conf['encoder_params']['pretrained'], type(conf['encoder_params']['pretrained']))
    
    snapshot_name = "{}{}_{}_{}_{}".format(prefix, conf["network"], conf["encoder_params"]["encoder"], fold, suffix)
    weights_path = os.path.join(weights_path, snapshot_name)
    print(weights_path)
    
    model = zoo.__dict__[conf["network"]](**conf["encoder_params"])
    model = torch.nn.DataParallel(model)
    print("=> loading checkpoint '{}''".format(weights_path))
    checkpoint = torch.load(weights_path, map_location="cpu")
    print("epoch", checkpoint["epoch"])
    model.load_state_dict(checkpoint["state_dict"])
    model.eval()
    if to_device: model.cuda()
    return model

models = []



# 0.80 version 34
# conf_path    = f"{github_folder}/configs/cls_ef_v9.json",
# weights_path = f"/kaggle/input/ef0-v9",
# prefix       = "ef_v9",


# 0.79 version 33
# conf_path    = f"{github_folder}/configs/cls_ef_v10.json",
#                                 weights_path = f"/kaggle/input/ef-v10",
#                                 prefix       = "ef_v10",


# 0.80 version 32, 2ndw 0.5 on fold 0
# model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v7.json",
#                                 weights_path = f"/kaggle/input/ef0-v7",
#                                 prefix       = "ef_v7",


# 0.80 version 30 full train with 2ndw 1
# conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
# weights_path = f"/kaggle/input/ef-v0-fulltrain",
# prefix       = "ef_v0",


# 0.78 version 29  no pretrain sed
# conf_path    = f"{github_folder}/configs/sed_ef_v0.json",
# weights_path = f"/kaggle/input/sed-ef-v0",
# prefix       = "sed_v0",

# 0.78 version 28   train 20 epoch on pretrain 123
# conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
# weights_path = f"/kaggle/input/ef-v0-pretrain123",

# 0.80 version 27   train 25 epoch on pretrain 123
# conf_path    = f"{github_folder}/configs/cls_ef_v6.json",
# weights_path = f"/kaggle/input/ef0-v6",
#                                 "ef_v6_e20",

# 0.79 version 26 better pretrain (21,22)
# conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
# weights_path = f"/kaggle/input/ef0-v0",
# prefix       = "ef_v0",

# 0.79 version 25 set 3 
# conf_path    = f"{github_folder}/configs/cls_ef2_v1.json",
# weights_path = f"/kaggle/input/ef2-v1",
# prefix       = "ef2_v1",

# 0.80 version 23-lb, 24-f1
#conf_path    = f"{github_folder}/configs/cls_ef2_v0.json",
#weights_path = f"/kaggle/input/ef2-v0",
#prefix       = "ef2_v0",


# 0.79 version 20
# conf_path    = f"{github_folder}/configs/cls_ef_v3.json",
# weights_path = f"/kaggle/input/ef0-v3",
# prefix       = "ef0_v3",


# rank 37 5folds version19
# conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
# weights_path = f"/kaggle/input/cls-ef-v0",
# prefix       = "ef_v0",

# only primary, 16min rank82  cls_ef_v0.json weights_path = f"/kaggle/input/cls-ef-v0", prefix = "ef_v0",
# rank90  cls_ef_v0.json weights_path = f"/kaggle/input/ef-v0-pretrain", prefix = "ef_v0_",


# 0.80 最差0.80 只有fold0  其他是上传错
# conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
#                                weights_path = f"/kaggle/input/e40-ef0",
#                                prefix       = "ef_v0",

# 44 min 0.79 240 conf_path    = f"{github_folder}/configs/cls_nf0_v5.json", weights_path = f"/kaggle/input/nf0-v5", prefix       = "nf0_v5_",
# 
#  cls_nf0_v5.json",weights_path = nfnet-baseline-bs16/nfnet-baseline-bs16, prefix= "nf0_v5_",


# 0.78
# f"{github_folder}/configs/cls_nf0_v5.json",
# weights_path = f"/kaggle/input/nfnet-baseline-bs16/nfnet-baseline-bs16",
# prefix       = "nf0_v5_",


## best only for now
suffixes = ["lb"]#, "f1_score" "last"]
folds    = [5, 10]

for i in folds:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v9.json",
                                weights_path = f"/kaggle/input/ef0-v9",
                                prefix       = "ef_v9",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

for i in folds:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v10.json",
                                weights_path = f"/kaggle/input/ef-v10",
                                prefix       = "ef_v10",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)
        
for i in folds:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v11.json",
                                weights_path = f"/kaggle/input/ef-v11",
                                prefix       = "ef_v11",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

for i in [10]:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
                                weights_path = f"/kaggle/input/effnet-b0-torch-avg-weights",
                                prefix       = "ef_v0",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)
            
# v30
for i in [5]:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v0.json",
                                weights_path = f"/kaggle/input/ef-v0-fulltrain",
                                prefix       = "ef_v0",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)

# v32
for i in [0]:
    for sx in suffixes:
        try:
            model = load_model(conf_path    = f"{github_folder}/configs/cls_ef_v7.json",
                                weights_path = f"/kaggle/input/ef0-v7",
                                prefix       = "ef_v7",
                               suffix       = sx,
                               fold         = i,
                               to_device    = False,)
            models.append(model)
        except Exception as e:
            print(f"model not found", e)            


len(models)
#models = [models[0]]

False <class 'bool'>
/kaggle/input/ef0-v9/ef_v9TimmClassifier_v3_tf_efficientnet_b0_5_lb
initing CLS features model 20 duration...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint '/kaggle/input/ef0-v9/ef_v9TimmClassifier_v3_tf_efficientnet_b0_5_lb''
model not found [Errno 2] No such file or directory: '/kaggle/input/ef0-v9/ef_v9TimmClassifier_v3_tf_efficientnet_b0_5_lb'
False <class 'bool'>
/kaggle/input/ef0-v9/ef_v9TimmClassifier_v3_tf_efficientnet_b0_10_lb
initing CLS features model 20 duration...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint '/kaggle/input/ef0-v9/ef_v9TimmClassifier_v3_tf_efficientnet_b0_10_lb''
epoch 17
False <class 'bool'>
/kaggle/input/ef-v10/ef_v10TimmClassifier_v3_tf_efficientnet_b0_5_lb
initing CLS features model 10 duration...
{'in_chans': 1, 'drop_path_rate': 0.2, 'drop_rate': 0.5}
=> loading checkpoint '/kaggle/input/ef-v10/ef_v10TimmClassifier_v3_tf_efficientnet_b0_5_lb''
model not found [Errn

6

In [7]:
import torch

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

f"{count_parameters(models[0])//1024/1024:.2f}M"

'3.51M'

## infer

In [8]:
CLASSES = "abethr1 abhori1 abythr1 afbfly1 afdfly1 afecuc1 affeag1 afgfly1 afghor1 afmdov1 afpfly1 afpkin1 afpwag1 afrgos1 afrgrp1 afrjac1 afrthr1 amesun2 augbuz1 bagwea1 barswa bawhor2 bawman1 bcbeat1 beasun2 bkctch1 bkfruw1 blacra1 blacuc1 blakit1 blaplo1 blbpuf2 blcapa2 blfbus1 blhgon1 blhher1 blksaw1 blnmou1 blnwea1 bltapa1 bltbar1 bltori1 blwlap1 brcale1 brcsta1 brctch1 brcwea1 brican1 brobab1 broman1 brosun1 brrwhe3 brtcha1 brubru1 brwwar1 bswdov1 btweye2 bubwar2 butapa1 cabgre1 carcha1 carwoo1 categr ccbeat1 chespa1 chewea1 chibat1 chtapa3 chucis1 cibwar1 cohmar1 colsun2 combul2 combuz1 comsan crefra2 crheag1 crohor1 darbar1 darter3 didcuc1 dotbar1 dutdov1 easmog1 eaywag1 edcsun3 egygoo equaka1 eswdov1 eubeat1 fatrav1 fatwid1 fislov1 fotdro5 gabgos2 gargan gbesta1 gnbcam2 gnhsun1 gobbun1 gobsta5 gobwea1 golher1 grbcam1 grccra1 grecor greegr grewoo2 grwpyt1 gryapa1 grywrw1 gybfis1 gycwar3 gyhbus1 gyhkin1 gyhneg1 gyhspa1 gytbar1 hadibi1 hamerk1 hartur1 helgui hipbab1 hoopoe huncis1 hunsun2 joygre1 kerspa2 klacuc1 kvbsun1 laudov1 lawgol lesmaw1 lessts1 libeat1 litegr litswi1 litwea1 loceag1 lotcor1 lotlap1 luebus1 mabeat1 macshr1 malkin1 marsto1 marsun2 mcptit1 meypar1 moccha1 mouwag1 ndcsun2 nobfly1 norbro1 norcro1 norfis1 norpuf1 nubwoo1 pabspa1 palfly2 palpri1 piecro1 piekin1 pitwhy purgre2 pygbat1 quailf1 ratcis1 raybar1 rbsrob1 rebfir2 rebhor1 reboxp1 reccor reccuc1 reedov1 refbar2 refcro1 reftin1 refwar2 rehblu1 rehwea1 reisee2 rerswa1 rewsta1 rindov rocmar2 rostur1 ruegls1 rufcha2 sacibi2 sccsun2 scrcha1 scthon1 shesta1 sichor1 sincis1 slbgre1 slcbou1 sltnig1 sobfly1 somgre1 somtit4 soucit1 soufis1 spemou2 spepig1 spewea1 spfbar1 spfwea1 spmthr1 spwlap1 squher1 strher strsee1 stusta1 subbus1 supsta1 tacsun1 tafpri1 tamdov1 thrnig1 trobou1 varsun2 vibsta2 vilwea1 vimwea1 walsta1 wbgbir1 wbrcha2 wbswea1 wfbeat1 whbcan1 whbcou1 whbcro2 whbtit5 whbwea1 whbwhe3 whcpri2 whctur2 wheslf1 whhsaw1 whihel1 whrshr1 witswa1 wlwwar wookin1 woosan wtbeat1 yebapa1 yebbar1 yebduc1 yebere1 yebgre1 yebsto1 yeccan1 yefcan yelbis1 yenspu1 yertin1 yesbar1 yespet1 yetgre1 yewgre1".split()
N_CLASS = len(CLASSES)

In [9]:
from torch.utils.data import Dataset, DataLoader

frame_length = 5
infer_frame_length = 5
sample_rate  = 32000

class TestDataset(Dataset):
    def __init__(self, 
                 df: pd.DataFrame, 
                 sample_rate:int,
                 frame_length:int,
                 waveform
                ):
        
        self.df = df
        
        self.frame_length = frame_length
        self.sample_rate = sample_rate
        
        self.waveform = waveform
        self.n_parts = math.ceil(len(waveform) / int(infer_frame_length * sample_rate))

    def __len__(self):
        return self.n_parts

    def __getitem__(self, idx: int):
        sample = self.df.loc[idx, :]
        row_id = sample.row_id
        sample_rate = self.sample_rate
        frame_length = self.frame_length

        end_seconds = int(sample.seconds)
        start_seconds = int(end_seconds - 5)
        
        y = self.waveform[sample_rate * start_seconds : sample_rate * end_seconds].astype(np.float32)
        
        wav_tensor = torch.from_numpy(y)
        if len(wav_tensor) == frame_length * sample_rate:
            pass
        elif len(wav_tensor) < frame_length * sample_rate:
            wav_tensor = torch.nn.functional.pad(wav_tensor, (0, (frame_length * sample_rate) - len(wav_tensor)))
        elif len(wav_tensor) > frame_length * sample_rate:
            wav_tensor = wav_tensor[:(frame_length * sample_rate)]
        
        # add channel
        wav_tensor = wav_tensor.unsqueeze(0)
        
        return {
            "wav_tensors": wav_tensor,
            "row_id": row_id,
        }


In [10]:
all_audios = list(Path("../input/birdclef-2023/test_soundscapes/").glob("*.ogg")) 
seconds = [i for i in range(5, 605, 5)]

class CFG:
    batch_size=4
    num_workers=4
    
config = CFG()

def prediction_for_clip(
    audio_path
):
    
    device = torch.device("cpu")
    
    # inference
    prediction_dict = {}
    
    clip, _ = librosa.load(audio_path, sr=sample_rate, mono=True)
    name_ = "_".join(audio_path.name.split(".")[:-1])
    row_ids = [name_+f"_{second}" for second in seconds]

    test_df = pd.DataFrame({
        "row_id": row_ids,
        "seconds": seconds
    })
    
        
    dataset = TestDataset(
        df=test_df, 
        sample_rate=sample_rate,
        frame_length=frame_length,
        waveform=clip
    )

    loader = DataLoader(
        dataset,
        batch_size=config.batch_size, 
        num_workers=config.num_workers,
        drop_last=False,
        shuffle=False,
        pin_memory=True
    )

    for data in loader:

        row_ids = data['row_id']

        for row_id in row_ids:
            if row_id not in prediction_dict:
                prediction_dict[str(row_id)] = []

        probas = []

        for model in models:
            with torch.no_grad():
                # print(type(data["wav_tensors"]))
                # print(data["wav_tensors"].size())
                # output = model(data["wav_tensors"])
                output = model(data["wav_tensors"], is_test=True)["logit"].sigmoid()
#                     
            for row_id_idx, row_id in enumerate(row_ids):
                prediction_dict[str(row_id)].append(output[[row_id_idx]].numpy().reshape(-1))
                                                        
    for row_id in list(prediction_dict.keys()):
                
        logits = np.array(prediction_dict[row_id]).mean(0)
        prediction_dict[row_id] = {}
        for label in range(N_CLASS):
            prediction_dict[row_id][CLASSES[label]] = logits[label]

    return prediction_dict

In [11]:
import concurrent.futures

start = time.time()
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    dicts = list(executor.map(prediction_for_clip, all_audios))
print(f"With concurrent ThreadPoolExecutor, time cost reduced to {time.time()-start:.1f} for processing {len(all_audios)} audios")


With concurrent ThreadPoolExecutor, time cost reduced to 54.5 for processing 1 audios


In [12]:
prediction_dicts = {}
for d in dicts:
    prediction_dicts.update(d)
    
submission = pd.DataFrame.from_dict(prediction_dicts, "index").rename_axis("row_id").reset_index()
submission.to_csv("submission.csv", index=False)
print(f"""
total rows : {len(submission)}
""")
submission.head()


total rows : 120



,row_id,abethr1,abhori1,abythr1,afbfly1,afdfly1,afecuc1,affeag1,afgfly1,afghor1,...,yebsto1,yeccan1,yefcan,yelbis1,yenspu1,yertin1,yesbar1,yespet1,yetgre1,yewgre1
0,soundscape_29201_5,0.053378,0.013911,0.027546,0.001620,0.003502,0.012571,0.012280,0.003905,0.011911,...,0.001206,0.001952,0.015644,0.004613,0.006087,0.014780,0.003944,0.003665,0.005191,0.007374
1,soundscape_29201_10,0.027837,0.063583,0.005887,0.027760,0.007703,0.009791,0.018649,0.006508,0.013148,...,0.001859,0.006037,0.036053,0.041102,0.013016,0.016755,0.011931,0.011364,0.005106,0.012909
2,soundscape_29201_15,0.036825,0.020818,0.037210,0.017486,0.005414,0.015562,0.015539,0.008906,0.008869,...,0.000936,0.005918,0.034370,0.006179,0.004029,0.023320,0.006574,0.014152,0.017586,0.020974
3,soundscape_29201_20,0.008926,0.017675,0.011207,0.016896,0.014212,0.010376,0.014910,0.008860,0.011449,...,0.001704,0.005474,0.024420,0.016400,0.005692,0.015152,0.022302,0.005707,0.008253,0.014152
4,soundscape_29201_25,0.011303,0.017986,0.008282,0.022437,0.013093,0.008577,0.019660,0.011060,0.012454,...,0.001701,0.005980,0.015507,0.017388,0.006887,0.014548,0.015489,0.005899,0.008831,0.012387
